# 0. Veri Setinin Hazırlanması

**Veri Seti ve Adresi:**

Forecasts for Product Demand: https://www.kaggle.com/datasets/felixzhao/productdemandforecasting

In [ ]:
# Kullanacağımız kütüphaneleri yükleyelim.

# Pandas ve NumPy
import pandas as pd
import numpy as np

# Görselleştirme kütüphaneleri
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Veri setini notebook içerisine dahil edelim.

df_ = pd.read_csv(filepath_or_buffer = "/kaggle/input/productdemandforecasting/Historical Product Demand.csv")
df = df_.copy()

In [ ]:
# Veri setinde rasgele eksik veriler oluşturalım.

import random

def add_random_missing_values(dataframe: pd.DataFrame,
                              missing_rate: float = 0.05,
                              seed: random = 42) -> pd.DataFrame:
    """Turns random values to NaN in a DataFrame.
    
    To use this function, you need to import pandas, numpy and random libraries.

    Args:
        dataframe (pd.DataFrame): DataFrame to be processed.
        missing_rate (float): Percentage of missing value rate in float format. Defaults 0.05

    
    """
    # Get copy of dataframe
    df_missing = dataframe.copy()

    # Obtain size of dataframe and number total number of missing values
    df_size = dataframe.size
    num_missing = int(df_size * missing_rate)
    
    # Set seed
    if seed:
        random.seed(seed)

    # Get random row and column indexes to turn them NaN
    for _ in range(num_missing):
        row_idx = random.randint(0, dataframe.shape[0] - 1)
        col_idx = random.randint(0, dataframe.shape[1] - 1)

        df_missing.iat[row_idx, col_idx] = np.nan
        
    return df_missing

df = add_random_missing_values(dataframe = df,
                               missing_rate = 0.03)

# 1. Veriye İlk Bakış

In [ ]:
# Veri setinin ilk 5 gözlemini görelim.
df.head()

In [ ]:
# Sondan 3 gözlemi görmek için tail() metodu kullanılır.
df.tail(3)

**Veri Setinde bulunan sütunlar şu bilgileri içermektedir:**
* Ürün kodu
* Depo
* Ürünün Kategorisi
* Satış Tarihi
* Sipariş Talebi

In [ ]:
# Veri setindeki kolonları yazdırmak için bu ifade kullanılmıştır.
print(list(df.columns), '\n')
# Veri setindeki satır ve sütun sayısını gösterir.
print(df.shape, '\n')
# Bu columnsların döndürdüğü veri tiplerini type ile yazdırılması.
print(f"df.columns'un döndüğü veri tipi: {type(df.columns)}\n")

In [ ]:
# İnfo metodu ile veri setinin yapısal bilgilerini görürüz.
df.info()

**df.info methodunun getirdiği çıktı ile bazı veri tiplerinin uygun olmadığını farkettim**. 
* Product_Code, Warehouse, ve Product_Category sütunları genelde kategorik ya da metin verisi içermeli.
* Date sütunu tarih verisi olmalı.
* Order_Demand sütunu ise sayısal veri (int veya float) olmalı .

In [ ]:
# Her sütundan örnek veriler görüntüleyerek veri tiplerini kontrol etme
print("Product_Code örnekleri:", df['Product_Code'].head(n=3))
print("Warehouse örnekleri:", df['Warehouse'].head(n=3))
print("Product_Category örnekleri:", df['Product_Category'].head(n=3))
print("Date örnekleri:", df['Date'].head(n=3))
print("Order_Demand örnekleri:", df['Order_Demand'].head(n=3))


* **Sütunların Veri Tiplerini Düzenleme:**

In [ ]:
# Product_Code, Warehouse, Product_Category sütunlarını kategorik sütunlara dönüştürme
categorical_columns = ['Product_Code', 'Warehouse', 'Product_Category']
df[categorical_columns] = df[categorical_columns].astype('category')

In [ ]:
# Tarih sütununu datetime formatına dönüştürme
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')  # Hatalı değerler NaT olacak

In [ ]:
# Veri tiplerini tekrar kontrol etmek.
print(df.info())

In [ ]:
# Order_Demand sütunundaki eşsiz değerleri kontrol etme
print(df['Order_Demand'].unique())

# Sayısal olmayan karakterler varsa temizleme
df['Order_Demand'] = df['Order_Demand'].str.replace('[^0-9]', '', regex=True).astype(float)


In [ ]:
# Sayısal sütunların istatistiksel özeti
print("\nSayısal Sütunların İstatistiksel Özeti:")

df.describe().T 

# 2. Eksik Veri Analizi

In [ ]:
# Veri setindeki eksik veri durumunu kontrol edin
df.isnull().sum()

In [ ]:
# Eksik veri oranını kontrol edin
df.isnull().sum() / len(df) * 100

In [ ]:
df.isna().sum().sum()

In [ ]:
# Eksik olmayan değerlerin sayısı
df.notnull().sum()

In [ ]:
# Veri setinde toplam kaç adet eksik gözlem var, kaç adet eksik olmayan gözlem var görelim.
print(f"Veri seti içerisinde toplam {df.notnull().sum().sum()} adet eksik olmayan, {df.isnull().sum().sum()} eksik gözlem var.")

In [ ]:
# Veri setinde en az bir gözlemi eksik olan kayıtlara da ulaşabiliriz.
df[df.isnull().any(axis = 1)]

In [ ]:

# Eksik veri ısı haritası
plt.figure(figsize=(10, 6))
sns.heatmap(df.isnull(), cbar=False, cmap="viridis")
plt.title("Eksik Veri Haritası")
plt.show()


In [ ]:
# Bir sayısal değişkenin dağılımını görmek için histogram kullanabiliriz.
plt.hist(df['Order_Demand'])
plt.show()

In [ ]:
import missingno as msno

msno.bar(df = df,
         figsize = (8, 4),
         fontsize = 10);


# 2.1 Eksik Verilerin Doldurulması


**Bu değerler, Order_Demand sütununda ciddi aykırı değerler olduğunu gösteriyor. Örneğin, medyan (300) ve ortalama (4905) arasında büyük bir fark var ve standart sapma çok yüksek.Aykırı değerlerin etkisi nedeniyle medyan yöntemi kullandım**


In [ ]:
df['Order_Demand'] = df['Order_Demand'].fillna(df['Order_Demand'].median())


In [ ]:
print(df['Order_Demand'].describe())


* Date sütunu tarihsel veri olduu için ileri ve geriye doldurma yöntemi ile doldurdum

In [ ]:
# İleriye doğru doldurma (Forward Fill)
df['Date'] = df['Date'].ffill()
df['Date'] = df['Date'].bfill()


Product_Category sütununun eksik veri oranı %2.95 ve veri tipi category olarak belirlenmiş. Kategorik veriler için eksik verileri doldurma işlemi, sayısal verilere kıyasla farklılık gösterir. Kategorik verilerde eksik değerleri doldururken genellikle en sık görülen kategori yöntemi tercih ettim.

In [ ]:
# En sık görülen kategori ile doldurma
mode_category = df['Product_Category'].mode()[0]
df['Product_Category'] = df['Product_Category'].fillna(mode_category)

In [ ]:
# En sık görülen değer (moda) ile doldurma
mode_product_code = df['Warehouse'].mode()[0]
df['Warehouse'] = df['Warehouse'].fillna(mode_product_code)


In [ ]:
# Gruplama yaparak en sık görülen Product_Code'u bul ve eksik değerleri doldur
df['Product_Code'] = df.groupby('Product_Category')['Product_Code'].transform(
    lambda x: x.fillna(x.mode()[0]) if not x.mode().empty else x
)

In [ ]:
# Eksik veri oranını kontrol edin
df.isnull().sum() / len(df) * 100

# 3. Kategorik Değişken Analizi

**Elimizdeki veri setinde kategorik olarak değerlendirilebilecek sütunlar şunlardır:**

* Product_Code
* Warehouse
* Product_Category

In [ ]:
# Kategorik sütunları seçelim
categorical_columns = ['Product_Code', 'Warehouse', 'Product_Category']

# Her bir kategorik sütunun benzersiz değer sayısını ve frekans dağılımını analiz etmek için yazılan kodlar.
for col in categorical_columns:
    print(f"--- {col} ---")
    print(f"Unique Values Count: {df[col].nunique()}")
    print(f"Value Counts:\n{df[col].value_counts().head(10)}")  # İlk 10 değeri göster
    print(f"Missing Values: {df[col].isnull().sum()} ({df[col].isnull().mean() * 100:.2f}%)\n")


In [ ]:

# Her bir kategorik değişken için barplot çizimi
for col in categorical_columns:
    plt.figure(figsize=(10, 6))
    sns.countplot(data=df, y=col, order=df[col].value_counts().index[:10])  # İlk 10 değer
    plt.title(f"Frekans Dağılımı: {col}")
    plt.xlabel("Kayıt Sayısı")
    plt.ylabel(col)
    plt.show()


Çubukların yüksekliği, o kategoriye ait kayıt sayısını temsil eder.
 * Product_Category için çok fazla tekrar eden kategorilerin popüler olduğu ve analizde odak noktası olduğunu göstermektedir.
* Product_Category grafiğine göre, A, B ve C kategorileri toplam kayıtların %60'ını oluşturuyor. Daha az temsil edilen kategoriler, detaylı analizlerde göz ardı edilebilir. 
* Warehouse grafiğinde, Depo_1 açık ara en fazla kayda sahipken Depo_5 oldukça düşük bir orana sahiptir. Depo kullanımındaki bu dengesizlik, lojistik verimlilik açısından incelenmelidir.

In [ ]:
# Çapraz tablo ile ilişki analizi
cross_tab = pd.crosstab(df['Product_Category'], df['Warehouse'])
print(cross_tab)

# Çapraz tablonun görselleştirilmesi
cross_tab.plot(kind='bar', stacked=True, figsize=(14, 8), colormap="viridis")
plt.title("Product_Category ve Warehouse İlişkisi")
plt.ylabel("Kayıt Sayısı")
plt.show()


**Kategoriler ve Depolar Arasındaki İlişki:**

* Belirli kategoriler sadece birka depoda yoğunlaşıyorsa stok yönetimi ve lojistik açısından önemli bri bilgidir. bu depo kritik bir rol oynar ve bu kategorinin diğer depolarada yayılması gerekmektedir.

**Anormallik Tespiti:**

* Product_Category ve Warehouse grafiğine göre, A kategorisi ağırlıklı olarak Depo_1 ve Depo_2'de bulunuyor. Ancak Depo_3'te bu kategoriye dair kayıt bulunmuyor. Bu, lojistik optimizasyon fırsatları için önemli bir bilgidir.Bazı kategoriler ise sadece bir depoda bulunuyor; bu durum tedarik zinciri açısından risk oluşturabilir.



In [ ]:
# Kategorik bir değişkendeki kategorilerin gözlem sayılarını görselleştirilmesi.
df['Product_Category'].value_counts().plot.barh();

In [ ]:
df['Warehouse'].value_counts().plot.barh();

In [ ]:
# Ortalama sipariş sayısını kategorilere göre gruplandırılması.
df.groupby('Product_Category')['Order_Demand'].mean().sort_values().plot(kind='bar', figsize=(10, 6))
plt.title('Product_Category bazında Ortalama Sipariş Sayısı')
plt.ylabel('Ortalama Sipariş')
plt.xlabel('Product_Category')
plt.show()


In [ ]:
df['Date'] = pd.to_datetime(df['Date'])
df.groupby([df['Date'].dt.to_period('M'), 'Product_Category'])['Order_Demand'].sum().unstack().plot(figsize=(12, 8))
plt.title('Product_Category Bazında Aylık Sipariş Değişimi')
plt.ylabel('Toplam Sipariş')
plt.xlabel('Tarih')
plt.show()


Bu analiz ile belirli dönemlerde hangi kategorilerin daha fazla talep gördüğünü belirlemek için kullanılabiliriz. Sezonluk trendler fark edilirse, talep tahmini ve stok yönetimi planlamasında yardımcı olabilir.

# 4. Sürekli Değişken Analizi

**Sürekli değişken analizi, veri setindeki sayısal sütunların (örneğin Order_Demand) davranışlarını anlamak ve özetlemek için yapılan bir adımdır.**

In [ ]:
# Veri setindeki numerik kolonları seçmek
df_numerical = df.select_dtypes(include = ["float64", "int64"])
df_numerical.head()

In [ ]:
df_numerical.describe().T

In [ ]:

# Dağılım grafiği oluşturma
plt.figure(figsize=(14, 6))
sns.histplot(df['Order_Demand'], bins=100, kde=True, color='blue')
plt.title('Order Demand Distribution', fontsize=16)
plt.xlabel('Order Demand', fontsize=14)
plt.ylabel('Frequency', fontsize=14)
plt.show()


**Grafik Özellikleri:**

* Yatay eksen (Order Demand): Sipariş talebi değerlerini gösteriyor.
* Dikey eksen (Frequency): Her bir sipariş talebi değerinin ne kadar sıklıkla gözlemlendiğini gösteriyor.

**Gözlemler:**

* Grafiğin sol tarafında büyük bir yoğunluk var. Bu, küçük sipariş taleplerinin (0'a yakın) oldukça yaygın olduğunu gösteriyor.Sipariş taleplerinin çoğu küçük değerlerden oluşurken, sağ tarafa doğru çok nadir ama çok büyük sipariş talepleri var (4 milyon gibi büyük değerler).

In [ ]:
# Kutu grafiği oluşturma
plt.figure(figsize=(14, 6))
sns.boxplot(x=df['Order_Demand'], color='lightblue')
plt.title('Boxplot of Order Demand', fontsize=16)
plt.xlabel('Order Demand', fontsize=14)
plt.show()



Order_Demand verilerini logaritmik dönüşüm gibi bir yöntemle küçük ve büyük değerlerin daha dengeli görünmesini sağladım.

In [ ]:
import numpy as np

# Negatif ve sıfır değerleri filtrelemek için verileri pozitif yapıyoruz (log10 sıfır ve negatif için tanımsızdır)
filtered_data = df[df['Order_Demand'] > 0]

# Logaritmik dönüşüm
filtered_data['Log_Order_Demand'] = np.log10(filtered_data['Order_Demand'])

# Logaritmik dönüşüm sonrası dağılım grafiği
plt.figure(figsize=(14, 6))
sns.histplot(filtered_data['Log_Order_Demand'], bins=50, kde=True, color='green')
plt.title('Log-Transformed Order Demand Distribution', fontsize=16)
plt.xlabel('Log(Order Demand)', fontsize=14)
plt.ylabel('Frequency', fontsize=14)
plt.show()



Logaritmik dönüşüm sayesinde veri ölçeği daha sıkışık hale gelmiş ve büyük sipariş taleplerinin etkisi azalmış. Bu, dağılımın daha dengeli görünmesini sağlamış.


Logaritmik dönüşümle birlikte, düşük ve orta seviyedeki sipariş taleplerinin daha belirgin bir şekilde yoğunlaştığı görülüyor (log değerleri 0 ile 3 arasında).


Sağ tarafa doğru azalan ancak halen gözlemlenebilen bir kuyruk var. Bu, büyük sipariş taleplerinin tamamen kaybolmadığını ancak etkisinin azaldığını gösteriyor.

In [ ]:
# Zaman serisine göre sipariş talebi toplamı
df['Date'] = pd.to_datetime(df['Date']) 
trend_data = df.groupby('Date')['Order_Demand'].sum()

plt.figure(figsize=(14, 6))
plt.plot(trend_data.index, trend_data.values, label='Order Demand Trend')
plt.title('Order Demand Over Time', fontsize=16)
plt.xlabel('Date', fontsize=14)
plt.ylabel('Total Order Demand', fontsize=14)
plt.legend()
plt.show()


1. Genel Gözlemler

* 2011 yılı sonlarına kadar sipariş talebi oldukça düşük görünüyor. Verinin başlangıç dönemi olduğu için veriler eksik olabilir veya operasyonlar henüz yeni başlamış olabilir.2012 yılından itibaren ise sipariş talebi hızlı bir şekilde artmış ve dalgalı bir şekilde yüksek seviyelerde devam etmiş. Bu, işletme büyümesinin veya talep artışının bir göstergesi olabilir.

2. Dalgalanma ve Pikler

* Grafik, her yıl boyunca düzenli dalgalanmalar ve talep artışlarını gösteriyor. Sipariş talebinde kısa dönemli ani artışlar görülebilir. Bu durum, belirli bir sezonda artan talebi gösterebilir.
  
* 2016 yılı sonlarına doğru sipariş talebinde bir düşüş gözlemleniyor. 


Düşüşün nedenlerini analiz etmek için, belirli bir tarih aralığını inceleyip diğer değişkenlerle (örneğin, ürün kategorisi, bölge veya satış) ilişkisi kontrol edildi.

In [ ]:


# Tarih sütununu datetime formatına çevir
df['Date'] = pd.to_datetime(df['Date'])

# 2016 ve sonrası verilerini seç
recent_data = df[df['Date'] >= '2016-01-01']

# 2016 boyunca aylık talep toplamı
monthly_trend = recent_data.groupby(recent_data['Date'].dt.to_period('M'))['Order_Demand'].sum()

# Görselleştir
import matplotlib.pyplot as plt

monthly_trend.plot(kind='bar', color='skyblue', figsize=(12, 6))
plt.title('2016 ve Sonrası Aylık Toplam Sipariş Talebi')
plt.ylabel('Toplam Talep')
plt.xlabel('Ay')
plt.xticks(rotation=45)
plt.show()



Bu grafikte, belirli bir tarih aralığında sipariş talebinin nasıl değiştiğini gözlenmektedir. Sezonsallık analizi ve zaman serisi modelleme yapmak için statsmodels veya Prophet gibi araçları kullanılabilir.


In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

# Tarihe göre sıralama ve toplam talep hesaplama
time_series = df.groupby('Date')['Order_Demand'].sum()
time_series = time_series.sort_index()

# Eksik verileri doldurma
time_series = time_series.resample('D').sum().fillna(0)  # Günlük veri

# Zaman serisini dekompoze et (trend, sezonsallık, residual)
result = seasonal_decompose(time_series, model='additive')

# Çıktıları görselleştir
result.plot()
plt.show()


# 5. Aykırı Değer Analizi (Outliers)



In [ ]:
df = df_.copy()

In [ ]:
df_Order_Demand = df['Order_Demand']
df_Order_Demand

In [ ]:
# Quantile değerlerin belirlenmesi.
Q1 = df_Order_Demand.quantile(0.25)
Q3 = df_Order_Demand.quantile(0.75)

print(Q1)
print(Q3)

# IQR değerin belirlenmesi.
IQR = Q3-Q1
print(IQR)

# Alt ve üst sınırların belirlenmesi.
lower_fence = Q1 - 1.5*IQR
upper_fence = Q3 + 1.5*IQR


Aykırı değerler alt sınır -2960.0'dan küçük ve üst sınır 5960.0'dan büyük olan değerlerdir.

In [ ]:
# Upper_fence üzerinde kalan aykırı gözlemlerin index değerlerini, daha sonra kullanmak üzere bir değişkende tutabiliriz.
outlier_idx = df_Order_Demand[df_Order_Demand > upper_fence].index

outlier_idx

**Aykırı Değerlerin Doldurulması**

In [ ]:
# Medyan hesaplama
median_value = df_Order_Demand.median()
print(f"Medyan Değer: {median_value}")


In [ ]:
# Aykırı değerleri belirlemek için
outliers = (df_Order_Demand < lower_fence) | (df_Order_Demand > upper_fence)

# Aykırı değerleri medyan ile doldurmak
df_Order_Demand_filled = df_Order_Demand.apply(
    lambda x: median_value if (x < lower_fence or x > upper_fence) else x
)

# Doldurulmuş veri setinin başını gösterelim
print(df_Order_Demand_filled.head())


In [ ]:


# Doldurulmuş veri seti için boxplot
sns.boxplot(data=df_Order_Demand_filled)
plt.title('Medyan ile Doldurulmuş Veri Seti - Aykırı Değerler')
plt.show()


In [ ]:
# Aykırı değerlerin sayısını hesaplamak
outlier_count = outliers.sum()
print(f"Aykırı Değer Sayısı: {outlier_count}")


In [ ]:
# Aykırı değerleri medyan ile doldurmak
df_filled = df_Order_Demand.apply(
    lambda x: median_value if (x < lower_fence or x > upper_fence) else x
)
print(f"Doldurulmuş Veri Seti: {df_filled.shape}")


# 6. Feature Engineering

In [ ]:
df.head()

In [ ]:
# Order_Demand üzerinden segmentasyon yapmak için fonksiyon
def demand_range(order_demand):
    if order_demand < 500:  # Düşük talepler için eşik
        return 'Low'
    elif 500 <= order_demand <= 20000:  # Orta talepler için eşik
        return 'Medium'
    else:  # Yüksek talepler için eşik
        return 'High'


df['demand_range'] = df['Order_Demand'].apply(demand_range)

# İlk 3 satırı görüntüleyelim
df.head()


In [ ]:

df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Day'] = df['Date'].dt.day
df['Weekday'] = df['Date'].dt.weekday  # Haftanın günü
df['Quarter'] = df['Date'].dt.quarter  # Çeyrek
df['Is_Weekend'] = df['Weekday'].isin([5, 6]).astype(int)  # Hafta sonu mu?
df.head(3)


In [ ]:
# Veriyi ürün bazında gruplama
df_grouped = df.groupby('Product_Category')['Order_Demand'].sum()

# Ürün bazında talep sıralaması
df_grouped = df_grouped.sort_values(ascending=False)

# Görselleştirme
df_grouped.head(10).plot(kind='bar', figsize=(10, 6))
plt.title('Ürün Bazında Toplam Talep')
plt.xlabel('Ürün')
plt.ylabel('Toplam Talep')
plt.show()


En çok talep alan ürünün stok seviyesini arttırabiliriz.

# Sonuç ve Öneriler


 **Taleplerin Tahmin Edilmesi**
 Order_Demand ve DATE  verisini kullanarak, gelecekteki taleplerin tahmin edilmesi sağlanmaktadır. Bu tahminler, işletmenin stok yönetimini optimize etmesine ve ürün tedarik süreçlerini daha verimli hale getirmesine yardımcı olur. Sezonsallık, trend, lag özellikleri gibi zaman serisi analizlerini kullanarak, gelecekteki talep artışları veya azalmaları öngörülebilir. Öneri: Modellemeyi, ARIMA, SARIMA gibi yöntemler ile gerçekleştirebilir.

 **Stok ve Envanter Yönetimi:**
Order_Demand ve Warehouse verisini analiz ederek, hangi ürünlerin daha fazla talep gördüğünü ve hangilerinin daha az talep aldığını belirlenebilir. Bu bilgiyi kullanarak, envanter yönetimi ve sipariş miktarlarını optimize edebilir, stok fazlası veya eksikliği azaltılabilir. Taleplerin sezonluk değişimleri göz önünde bulundurularak, stok seviyeleri ayarlanabilir.

 **Ürün Segmentasyonu ve Satış Stratejileri:**
ürünlerin pazarlanması ve fiyatlandırılması stratejik olarak belirlenebilir.
Order_Demand verisini kullanarak, ürünleri düşük, orta ve yüksek talep gruplarına ayırarak bu segmentlere göre, her ürün için özelleştirilmiş pazarlama stratejileri geliştirilebilir. Yüksek talep gören ürünler için promosyonlar düzenlenebilir ve düşük talep gören ürünleri indirimle tanıtılabilir.